In [1]:
import azureml.core
from azureml.core import Experiment, Workspace, Dataset, Datastore, ScriptRunConfig, Run
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
import os
import shutil
import glob
import math
from os.path import join
import tensorflow as tf
from sys import path
import numpy as np

# check core SDK version number

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.41.0


In [2]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
# get metadata about the workspace
print(ws.name, ws.location, ws.resource_group, sep='\t')
# list the registered datastores
ws.datastores

solar	eastus	cic_ai


{'solardatablob': {
   "name": "solardatablob",
   "container_name": "solar",
   "account_name": "aiprojects",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceblobstore': {
   "name": "workspaceblobstore",
   "container_name": "azureml-blobstore-8177f378-98da-4ec7-beac-2cce3eb1e5be",
   "account_name": "aiprojects",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceartifactstore': {
   "name": "workspaceartifactstore",
   "container_name": "azureml",
   "account_name": "aiprojects",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceworkingdirectory': {
   "name": "workspaceworkingdirectory",
   "container_name": "codeshare-8177f378-98da-4ec7-beac-2cce3eb1e5be",
   "account_name": "aiprojects",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspacefilestore': {
   "name": "workspacefilestore",
   "container_name": "azureml-filestore-8177f378-98da-4ec7-beac-2cce3eb1e5be",
   "account_name": "aiproje

In [3]:
# access our registered data share containing image data in this workspace
datastore = Datastore.get(workspace = ws, datastore_name = 'solardatablob')

cpk_train_path = (datastore, 'CPK_solar/s1s2data/training/')
cpk_eval_path = (datastore, 'CPK_solar/s1s2data/eval/')

# nc_train_path = (datastore, 'NC_solar/training/')
# nc_eval_path = (datastore, 'NC_solar/eval/')

test_path = (datastore, 'CPK_solar/s1s2data/predict/testpred5')

train_dataset = Dataset.File.from_files(path = [cpk_train_path])
eval_dataset = Dataset.File.from_files(path = [cpk_eval_path])

# nc_train_dataset = Dataset.File.from_files(path = [nc_train_path])
# nc_eval_dataset = Dataset.File.from_files(path = [nc_eval_path])

# when we combine datasets the selected directories and relative paths to the datastore are brought in
# mount folder
# |-cddatafilestore
# | |-GEE
# | | |-training
# | | |-eval
# | |-Onera
# | | |-training
# | | |-eval

# train_dataset = Dataset.File.from_files(path = [cpk_train_path, nc_train_path])
# eval_dataset = Dataset.File.from_files(path = [cpk_eval_path, nc_eval_path])
test_dataset = Dataset.File.from_files(path = [test_path])

In [4]:
# define the compute target
ws.compute_targets
compute_target = ws.compute_targets['mevans-gpu']

In [5]:
# get a previously working environment
exp = Experiment(workspace = ws, name = 'solar-nc-cpk')
run = Run(experiment = exp, run_id = 'solar-nc-cpk_1643057518_98d03e88')
env = run.get_environment()

In [6]:
experiment_name = 'solar-s1s2'
exp = Experiment(workspace = ws, name = experiment_name)

In [27]:
envs = Environment.list(workspace = ws)
env = envs.get('AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu')

In [39]:
# add rasterio to our environment
curated_clone = env.clone("customize_curated")
conda_dep = CondaDependencies()
# Installs numpy version 1.17.0 conda package
conda_dep.add_conda_package("rasterio")

# Installs pillow package
# conda_dep.add_pip_package("pillow")

# Adds dependencies to PythonSection of myenv
curated_clone.python.conda_dependencies=conda_dep
curated_clone.register()

In [7]:
# attach our utilities folder to the path to import modules
path.append('/home/azureuser/cloudfiles/code/Users/mevans/Solar_UNet/azure/scv/utils')

In [8]:
from model_tools import get_binary_model, make_confusion_matrix
from processing import get_training_dataset

In [9]:
# Define some global variabes

BANDS = ["B2", "B3", "B4", "B8", "B11", "B12", "VV", "VH", "off_nadir"]
RESPONSE = 'landcover'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

In [10]:
# Create mountcontext and mount the dataset
test_mount = train_dataset.mount()

# open access to the mount point
test_mount.start() 

# Get the mount point
dataset_mount_folder = test_mount.mount_point
print(dataset_mount_folder)

test_files = []
for root, dirs, files in os.walk(dataset_mount_folder):
    for f in files:
        test_files.append(join(root, f))

# test_files = glob.glob(join(dataset_mount_folder, '*.'))

# read in the training data so we can calculate size

training = get_training_dataset(
    files = test_files,
    ftDict = FEATURES_DICT,
    features = BANDS,
    response = RESPONSE,
    buff = 1,
    batch = 1,
    repeat = False)

/tmp/tmpcyaaih2g


In [41]:
# iterator = iter(training)
example = iterator.next()
tf.reduce_min(example[0][:,:,:,-3])

<tf.Tensor: shape=(), dtype=float32, numpy=-1.4141586>

In [42]:
test_mount.stop()

In [76]:
# # calculate some summary statistics used in model training
# # calculate some summary statistics used in model training
solarPix = 0
nonPix = 0

iterator = iter(training)
i = 0
for example in iterator:
    ones = tf.reduce_sum(example[1])
    zeroes = (256*256) - ones
    solarPix += ones
    nonPix += zeroes
    i+=1

# m = get_model(depth = len(BANDS), optim = tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999), loss = 'mse', mets = [tf.keras.metrics.categorical_accuracy], bias = None)
# train_con_mat = make_confusion_matrix(training, m)
BIAS = np.log(solarPix/nonPix)#np.log(classums[1]/classums[0])
WEIGHT = nonPix/solarPix
TRAIN_SIZE = i#(nonPix+solarPix)/(256*256)#train_con_mat.sum()//(256*256)

print('size = ', TRAIN_SIZE)
print(f'bias = {BIAS}')
print(f'weight = {WEIGHT}')

test_mount.stop()

size =  3833
bias = -3.657029151916504
weight = 38.746063232421875


In [17]:
print('size', TRAIN_SIZE)
print('weight', WEIGHT)
print('bias', BIAS)

size 3833
weight tf.Tensor(38.746063, shape=(), dtype=float32)
bias -3.6570292


In [10]:
# use the azure folder as our script folder
source = 'Satellite_ComputerVision'
util_folder = 'utils'
script_folder = f'{source}/azure'
script_file = 'train_solar.py'

In [ ]:
# copy the entire 'utils' folder from github repo to local
# shutil.copytree(src = f'/home/azureuser/cloudfiles/code/Users/mevans/Satellite_ComputerVision/{script_folder}',
# dst = f'/home/azureuser/cloudfiles/code/Users/mevans/{script_folder}')

# shutil.copytree(src = f'/home/azureuser/cloudfiles/code/Users/mevans/Satellite_ComputerVision/{util_folder}',
# dst = f'/home/azureuser/cloudfiles/code/Users/mevans/{script_folder}/{util_folder}')

In [23]:
# create script run config
# WEIGHT = int(math.ceil(WEIGHT/5)*5)
# define the command line arguments to our training sript
args = [
    '--train_data', train_dataset.as_mount(),
    '--eval_data', eval_dataset.as_mount(),
    '--test_data', test_dataset.as_mount(),
    '--weight', 120,
    '--bias', 0,#BIAS,
    '-lr', 0.001,
    '--epochs', 100,
    '--batch', 16,
    '--size', 3833,
    '--kernel_size', 256,
    '--response', f'{RESPONSE}',
    '--bands', '["B2", "B3", "B4", "B8", "B11", "B12", "VV", "VH", "off_nadir"]',
    '--splits', '[6]']

src = ScriptRunConfig(source_directory='azure',
                      script='train_solar.py', 
                      arguments=args,
                      compute_target=compute_target,
                      environment=env)

In [24]:
#  run the training job
run = exp.submit(config=src, tags = dict({'model':'Unet', 'dataset':'CPK+NC s1s2', 'normalization':'pixel', 'epochs':'0-100'}))
run

Experiment,Id,Type,Status,Details Page,Docs Page
solar-s1s2,solar-s1s2_1655827397_96310180,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [19]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…